In [ ]:
# default_exp projects.self_supervised

# Self-supervised Learning
> Self-supervised utility code

In [ ]:
# hide
from nbdev.showdoc import *
from fastcore.test import *

%load_ext autoreload
%autoreload 2

In [ ]:
# export
# import pandas as pd
# import numpy as np

from fastai.vision.all import *
from self_supervised.layers import *
from self_supervised.models.vision_transformer import *
from scp.utils.general import custom_load

import torchvision
import skimage
import colorsys
from skimage import io
from skimage.measure import find_contours
from matplotlib.patches import Polygon
import cv2
from vit_pytorch.vit import Transformer
from einops.layers.torch import Rearrange
from einops import repeat
from torch import nn
from fastai.callback.core import *

## Model splitter functions

Split the model architecture at certain points to allow for discriminative learning rates. 

In [ ]:
# export
def model_split1(model):
    '''Basic feature extractor & FC head split for '''
    groups = L([model.vit_backbone, model.mlp])
    return groups.map(params)

def model_split2(model):
    '''Attempt to do a split like in fastai i.e. feature extractor is split in the middle'''
    g1 = nn.Sequential(model.vit_backbone.patch_embed, model.vit_backbone.pos_drop, model.vit_backbone.blocks[:6])
    g2 = nn.Sequential(model.vit_backbone.blocks[6:], model.vit_backbone.norm, model.vit_backbone.head)
    groups = L([g1, g2, model.mlp])
    return groups.map(params)

In [ ]:
# export
# https://keremturgutlu.github.io/self_supervised/
# copy and pasted directly from:
# https://github.com/KeremTurgutlu/self_supervised/blob/main/examples/vision/06%20-%20training_dino_iwang.ipynb
class ViTClassifier(Module):
    def __init__(self, vit_backbone, n_feat_layers, n_classes, lin_f=1024, lin_drop=0.3, pooling='avg'):
        self.vit_backbone  = vit_backbone
        self.n_feat_layers = n_feat_layers 
        self.pooling = pooling
        out_dim = self.vit_backbone.norm.weight.size(0)
        
        if self.n_feat_layers == 1: in_f = 2*out_dim
        else:
            if pooling == 'avg':   in_f = out_dim
            elif pooling == 'cat': in_f = out_dim*n_feat_layers
        
        self.mlp = create_cls_module(in_f, n_classes)
        
    def forward(self,x):
        
        out = self.vit_backbone.get_intermediate_layers(x,self.n_feat_layers)
        
        if self.n_feat_layers == 1:
            # cat [CLS] token and avgpooled output tokens from the last layer
            cls_token, output_tokens = out[0][:,0],out[0][:,1:]
            x = torch.cat([cls_token, output_tokens.mean(1)], dim=1)
        else:
            # avgpool or cat [CLS] tokens from last n layers
            out = [o[:,0] for o in out] 
            if self.pooling == 'avg':   x = torch.stack(out,dim=0).mean(0)
            elif self.pooling == 'cat': x = torch.cat(out, 1)
            else:                       raise Exception("Pooling should be avg or cat")
                
        return self.mlp(x)

In [ ]:
# export
def load_state_dict(arch, pretraining, state_dict_path:str=None, **kwargs):
    '''Get pretrained arch'''
    if arch == "resnet50":
        if pretraining == "sl-imagenet":
            return resnet50(pretrained=True).state_dict()
        elif pretraining == "dino-imagenet":
            # https://github.com/facebookresearch/dino
            return torch.hub.load('facebookresearch/dino:main', 'dino_resnet50').state_dict()
        elif pretraining == "dino-custom":
            dino_state_dict = custom_load(state_dict_path, **kwargs)
            state_dict = OrderedDict()
            for k,v in dino_state_dict.items():
                if "teacher" not in k and k!="C":
                    state_dict[k] = v
            return state_dict
        else:
            raise Exception(f"Unknown pretraining '{arch}'")
            
    elif arch == "deit_small":
        if pretraining == "sl-imagenet":
            # https://github.com/facebookresearch/deit
            return torch.hub.load('facebookresearch/deit:main', 'deit_small_patch16_224', pretrained=True).state_dict()
        elif pretraining == "dino-imagenet":
            # https://github.com/facebookresearch/dino
            return torch.hub.load('facebookresearch/dino:main', 'dino_vits16').state_dict()
        elif pretraining == "dino-custom":
            dino_state_dict = custom_load(state_dict_path, **kwargs)
            state_dict = OrderedDict()
            for k,v in dino_state_dict.items():
                if "teacher" not in k and k!="C":
                    state_dict[k] = v
            return state_dict
        else:
            raise Exception(f"Unknown pretraining '{arch}'")
            
    else:
        raise Exception(f"Unknown architecture '{arch}'")

In [ ]:
# export
def get_dino_arch(arch:str, **kwargs):
    '''We do not care about weights here, simply get the model architecture'''

    if arch == "deit_small": # 21M params
        model = deit_small(**kwargs) # naming artefact (dino fastai library and dino FB library)
    elif arch == "resnet50":
        model = resnet50()
        model.embed_dim = model.fc.weight.shape[1]
        model.fc = Identity()
    else:
        raise Exception(f"Unknown architecture '{arch}'")
        
    return MultiCropWrapper(model)

def get_opt_func(optimizer:str):
    '''get optimizer'''
    if optimizer=="adam":
        return Adam
    elif optimizer=="sgd":
        return SGD
    else:
        raise Exception(f"No optimizer found that matches '{optimizer}'")

## DINO Visualizations


In [ ]:
# export 
# code taken from https://github.com/facebookresearch/dino/blob/main/visualize_attention.py
def apply_mask(image, mask, color, alpha=0.5):
    for c in range(3):
        image[:, :, c] = image[:, :, c] * (1 - alpha * mask) + alpha * mask * color[c] * 255
    return image

def random_colors(N, bright=True):
    """
    Generate random colors.
    """
    brightness = 1.0 if bright else 0.7
    hsv = [(i / N, 1, brightness) for i in range(N)]
    colors = list(map(lambda c: colorsys.hsv_to_rgb(*c), hsv))
    random.shuffle(colors)
    return colors

def display_instances(image, mask, fname="test", figsize=(5, 5), blur=False, contour=True, alpha=0.5):
    fig = plt.figure(figsize=figsize, frameon=False)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax = plt.gca()

    N = 1
    mask = mask[None, :, :]
    # Generate random colors
    colors = random_colors(N)

    # Show area outside image boundaries.
    height, width = image.shape[:2]
    margin = 0
    ax.set_ylim(height + margin, -margin)
    ax.set_xlim(-margin, width + margin)
    ax.axis('off')
    masked_image = image.astype(np.uint32).copy()
    for i in range(N):
        color = colors[i]
        _mask = mask[i]
        if blur:
            _mask = cv2.blur(_mask,(10,10))
        # Mask
        masked_image = apply_mask(masked_image, _mask, color, alpha)
        # Mask Polygon
        # Pad to ensure proper polygons for masks that touch image edges.
        if contour:
            padded_mask = np.zeros((_mask.shape[0] + 2, _mask.shape[1] + 2))
            padded_mask[1:-1, 1:-1] = _mask
            contours = find_contours(padded_mask, 0.5)
            for verts in contours:
                # Subtract the padding and flip (y, x) to (x, y)
                verts = np.fliplr(verts) - 1
                p = Polygon(verts, facecolor="none", edgecolor=color)
                ax.add_patch(p)
    ax.imshow(masked_image.astype(np.uint8), aspect='auto')
    fig.savefig(fname)
    return

In [ ]:
# export 


class MAECustom(nn.Module):
    '''Adapted MAE implementation from https://github.com/lucidrains/vit-pytorch implementation
    
    This MAE can be initialized with Timm and FB ViTs, thus taking advantage of pretraining.
    The loss is also not calculated in the forward() function anymore. 
    Insted the preds and ground truths are returned (instead of the loss)
    '''
    def __init__(
        self,
        *,
        encoder,
        decoder_dim,
        masking_ratio = 0.75,
        decoder_depth = 1,
        decoder_heads = 8,
        decoder_dim_head = 64
    ):
        super().__init__()
        assert masking_ratio > 0 and masking_ratio < 1, 'masking ratio must be kept between 0 and 1'
        self.masking_ratio = masking_ratio

        # extract some hyperparameters and functions from encoder (vision transformer to be trained)

        self.encoder = encoder
        
        channel_num = self.encoder.patch_embed.proj.weight.shape[1]
        
        num_patches, encoder_dim = self.encoder.patch_embed.num_patches+1, encoder.embed_dim      
        
        # recreate 'to_patch' function from https://github.com/lucidrains/vit-pytorch implementation
        if isinstance(self.encoder.patch_embed.patch_size, int):
            self.encoder.patch_embed.patch_size = (self.encoder.patch_embed.patch_size, self.encoder.patch_embed.patch_size)
        patch_height, patch_width = self.encoder.patch_embed.patch_size
        self.to_patch =  Rearrange('b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = patch_height, p2 = patch_width)
        pixel_values_per_patch = self.encoder.patch_embed.patch_size[0]**2 * channel_num
    
        # decoder parameters

        self.enc_to_dec = nn.Linear(encoder_dim, decoder_dim) if encoder_dim != decoder_dim else nn.Identity()
        self.mask_token = nn.Parameter(torch.randn(decoder_dim))
        self.decoder = Transformer(dim = decoder_dim, depth = decoder_depth, heads = decoder_heads, dim_head = decoder_dim_head, mlp_dim = decoder_dim * 4)
        self.decoder_pos_emb = nn.Embedding(num_patches, decoder_dim)
        self.to_pixels = nn.Linear(decoder_dim, pixel_values_per_patch)

    def forward(self, img):
        device = img.device
        
        # get patches
        
        patches = self.to_patch(img)
        batch, num_patches, *_ = patches.shape

        # img to encoder tokens and add positions

        tokens = self.encoder.patch_embed(img)
        tokens = tokens + self.encoder.pos_embed[:, 1:(num_patches + 1)]
        
        # calculate of patches needed to be masked, and get random indices, dividing it up for mask vs unmasked

        num_masked = int(self.masking_ratio * num_patches)
        rand_indices = torch.rand(batch, num_patches, device = device).argsort(dim = -1)
        masked_indices, unmasked_indices = rand_indices[:, :num_masked], rand_indices[:, num_masked:]

        # get the unmasked tokens to be encoded

        batch_range = torch.arange(batch, device = device)[:, None]
        tokens = tokens[batch_range, unmasked_indices]

        # get the patches to be masked for the final reconstruction loss

        masked_patches = patches[batch_range, masked_indices]

        # attend with vision transformer

        encoded_tokens = self.encoder.blocks(tokens)

        # project encoder to decoder dimensions, if they are not equal - the paper says you can get away with a smaller dimension for decoder

        decoder_tokens = self.enc_to_dec(encoded_tokens)

        # repeat mask tokens for number of masked, and add the positions using the masked indices derived above

        mask_tokens = repeat(self.mask_token, 'd -> b n d', b = batch, n = num_masked)
        mask_tokens = mask_tokens + self.decoder_pos_emb(masked_indices)

        # concat the masked tokens to the decoder tokens and attend with decoder

        decoder_tokens = torch.cat((mask_tokens, decoder_tokens), dim = 1)
        decoded_tokens = self.decoder(decoder_tokens)

        # splice out the mask tokens and project to pixel values

        mask_tokens = decoded_tokens[:, :num_masked]
        pred_pixel_values = self.to_pixels(mask_tokens)

        # calculate reconstruction loss
        return (pred_pixel_values, masked_patches)
    
class MAECallback(Callback):
    '''Make MAE output compatible to fastai's training loop
    
    Semi-official MAE implementation returns not just predictions but also the ground truth. 
    Fastai does not expect this, therefore this needs to be split before loss is calculated. 
    '''
    
    def after_pred(self):
        '''Split MAE model output into predicition and ground truth'''
        self.learn.yb = (self.learn.pred[1],) # yb only supports tuple assignement
        self.learn.pred = self.learn.pred[0]